# **Particle Swarm optimization**
class Particle:
- Define the particle object with the **__init__**. Every particle must have a position, a velocity.
- Define a function, called **FitnessCalculator**, that computes the fitness value of the particle. Its indipendent variable is the **accuracy metric** of the neural network.
- Define a function, called **PositionCalculator**
- Define a function, called **VelocityCalculator**


In [6]:
import numpy as np
import random

In [7]:
class Particle:

    def __init__(self, position, velocity ):
        '''position and velocity are vectors (lists)'''

        self.position = position
        self.velocity = velocity
        self.bestp = position

        self.iteration = 0


    def FitnessCalculator(self, position, accuracy):
        '''It takes as input the model and the parameter (which isthe particle position).
         Calculates the accuracy (or loss, we need to decide) of the model and return it. '''

        # Ho messo model.accuracy, poi quando definiamo il modello credo qui vada messa la parte proprio del training per ritornare poi l'accuracy o la loss in base a come vogliamo fare noi.

        self.fitness = accuracy(position)
        return self


    def VelocityCalculator(self, c1, c2, best_glob_pos, v_max):

        #np.random.seed(3)

        random_1 = np.random.random(len(self.position))
        random_2 = np.random.random(len(self.position))

        w = c1*random_1 + c2*random_2

        velocity = w*self.velocity + c1*random_1*(self.bestp - self.position) + c2*random_2*(best_glob_pos - self.position)

        # velocity quickly explodes to large values, especially for particles far from the neighborhood best and personal best positions. Consequently, particles have large position updates, which result in particles leaving the boundaries of the search space – the particles diverge. To control the global exploration of particles, velocities are clamped to stay within boundary constraints.

        new_velocity = np.zeros(len(velocity)) #I inizialize an array with all zeros and then I change the elements

        for i in range (len(velocity)):
            if velocity[i] > v_max[i]:
                new_velocity[i] = v_max[i]
            else:
                new_velocity[i] = velocity[i]
        self.velocity = new_velocity




    def PositionCalculator(self, new_vel):
        
        self.iteration += 1

        self.position = self.position + new_vel
        return self

    def BestLocal(self, problem):
        '''Takes as input the particle and the type of optimization problem (problem could be minimum or maximum) and calculates best fitness and best position'''
        if self.iteration == 0:
            self.bestfit = self.fitness

        if problem == 'minimum':
            if self.fitness < self.bestfit:
                self.bestfit = self.fitness
                self.bestp = self.position
        elif problem == 'maximum':
            if self.fitness > self.bestfit:
                self.bestfit = self.fitness
                self.bestp = self.position
        else:
            return "Error! problem must be: 'minimum' or 'maximum'"    
        return self

In [8]:
def InitializeSwarm(swarm_size, dimensionality, lower_bound, upper_bound):
    '''It takes as input the swarm size (number of particles I want to create, the dimensionality of the swarm (number of parameter for each particle) and the lower and upper bound of the parameter (that are two lists))'''

    #np.random.seed(3)
    # Usually the positions of particles are initialized to uniformly cover the search space
    swarm_list = []
    for particle in range(swarm_size):
        position = np.random.uniform(lower_bound, upper_bound, dimensionality)

        velocity = np.zeros(dimensionality)
        # velocity = np.random.random(dimensionality)

        part = Particle(position, velocity)
        swarm_list.append(part)

    return swarm_list
#________________________________________________________

lower_bound = [0,0,0]
upper_bound = [10,10,10]
swarm = InitializeSwarm(10,3, lower_bound, upper_bound)

for particle in swarm:
    print(particle.position, particle.velocity)

[8.12386372 4.06567851 8.5176156 ] [0. 0. 0.]
[9.81896327 8.00028974 5.91630802] [0. 0. 0.]
[3.90405824 0.31632141 8.05148766] [0. 0. 0.]
[5.53396717 6.05945355 6.4010241 ] [0. 0. 0.]
[5.00162502 4.36279368 4.33141519] [0. 0. 0.]
[5.07239342 6.34169204 2.52528769] [0. 0. 0.]
[6.18352207 3.32393751 0.91997515] [0. 0. 0.]
[4.16998259 9.23242423 8.86475467] [0. 0. 0.]
[8.65315008 0.93319208 1.59408663] [0. 0. 0.]
[6.35240132 1.44443453 5.6778241 ] [0. 0. 0.]


In [9]:
# I create a swarm of 5 particles, each one with 3 dimensions
    # swarm = InitializeSwarm(5,3)
    
    # After inizializing the swarm we need to evaluate each particle
    
    # Let's first define the function we want to use for the evaluation:
def f(lista):
    '''Definisco una funzione f che prende in input una lista di tre elementi (ovviamente va cambiata se cambio la swarm size)'''
    return(lista[0]**2+lista[1]**2+lista[2]**2)

for particle in swarm:
    particle.FitnessCalculator(particle.position, f)

    # When we inizialize the swarm we need also to calculate the local optimum:
    particle.BestLocal(problem = 'minimum')

    print('New particle \n The particle fitness is: ',particle.fitness, 'The particle best position is: ',particle.bestp, 'The particle best fit is: ', particle.bestfit, '\n')

New particle 
 The particle fitness is:  155.0766791010892 The particle best position is:  [8.12386372 4.06567851 8.5176156 ] The particle best fit is:  155.0766791010892 

New particle 
 The particle fitness is:  195.4193762010904 The particle best position is:  [9.81896327 8.00028974 5.91630802] The particle best fit is:  195.4193762010904 

New particle 
 The particle fitness is:  80.16818351243333 The particle best position is:  [3.90405824 0.31632141 8.05148766] The particle best fit is:  80.16818351243333 

New particle 
 The particle fitness is:  108.31487948315035 The particle best position is:  [5.53396717 6.05945355 6.4010241 ] The particle best fit is:  108.31487948315035 

New particle 
 The particle fitness is:  62.81137906271486 The particle best position is:  [5.00162502 4.36279368 4.33141519] The particle best fit is:  62.81137906271486 

New particle 
 The particle fitness is:  72.32331085417371 The particle best position is:  [5.07239342 6.34169204 2.52528769] The par

In [10]:
# Let's find the global optimum found until now
def global_optimum(swarm):
    '''takes as input a swarm of particles and return the position of global optimum and the value of the fitness function in that global optimum'''

    global_opt = (min([particle.fitness for particle in swarm]))
    best_global_position = swarm[np.argmin(np.array([particle.fitness for particle in swarm]))].position

    return best_global_position, global_opt    

#_________________________________________________________________

best_global_position, global_opt = global_optimum(swarm)

#print('The best global position is: ',best_global_position, 'The best global fitness is: ', global_opt )

#____________________________________________________________________

# Now I have to iterate and update the best known local and global position
# First let's set the parameter for the velocity calculation

def acceleration_coefficient(lower_bound, upper_bound):
    '''It takes as input two vector: lower_bound is a vector containing the minimum element which can assume each parameter and upper_buond contains the maximum. e.g. for parameter j which assume value in range (aj,bj), the j-esimo element of lower_bound will be aj and the j-esimo element of upper_bund will be bj'''

    #random.seed(3)
    a1 = random.random()
    a2 = random.random()

    c1 = a1*(np.array(upper_bound)-np.array(lower_bound))
    c2 = a2*(np.array(upper_bound)-np.array(lower_bound))

    return c1,c2

c1, c2 = acceleration_coefficient(lower_bound,upper_bound)

# w = 0.9
criteria_not_reach = True
iteration = 0 #CREDO CHE POTREI UTILIZZARE DIRETTAMENTE SELF.ITERATION

#print('c1: ', c1, '\n c2: ', c2)

while criteria_not_reach:
    #print('ITERATION: ', iteration)

    # First we update the new position
    for particle in swarm:
        ## We need to calculate the new velocity:",
        particle.VelocityCalculator(c1, c2, best_global_position, v_max = [3,3,3])

        ## Now we can calculate the new position:\n",
        particle.PositionCalculator(particle.velocity)

        ## We need to evaluate again the fitness function\n",
        particle.FitnessCalculator(particle.position, f)

        ## With the new position calculated we have to update the local best position:
        particle.BestLocal(problem = 'minimum')

        #print('New velocity: ', particle.velocity, '\n New position: ', particle.position, '\n New fitness: ', particle.fitness , '\n Best local position: ', particle.bestp, 'Best local fitness', particle.bestfit ,'\n')

        #print('ANOTHER PARTICLE \n')

        # w /= 2 #at each iteraton w is halved (IN THIS CASE I HAVE SET W TO A FIXED VALUE DIRECTLY INTO THE CLASS PARTICLE )
        #print('w: ', w)

    # Then we update the global optimum
    best_position, opt = global_optimum(swarm)
    if opt< global_opt:
        global_opt = opt
        best_global_position = best_position
    
    print('GLOBAL OPTIMUM: ', global_opt, 'GLOBAL OPT POSITION: ', best_global_position)

    iteration += 1
    if iteration == 200:
        criteria_not_reach = False

GLOBAL OPTIMUM:  41.24889245345586 GLOBAL OPT POSITION:  [5.03424084 3.93319208 0.65978159]
GLOBAL OPTIMUM:  41.24889245345586 GLOBAL OPT POSITION:  [5.03424084 3.93319208 0.65978159]
GLOBAL OPTIMUM:  41.24889245345586 GLOBAL OPT POSITION:  [5.03424084 3.93319208 0.65978159]
GLOBAL OPTIMUM:  36.951539379329894 GLOBAL OPT POSITION:  [5.76297535 0.11036479 1.93066676]
GLOBAL OPTIMUM:  36.951539379329894 GLOBAL OPT POSITION:  [5.76297535 0.11036479 1.93066676]
GLOBAL OPTIMUM:  36.951539379329894 GLOBAL OPT POSITION:  [5.76297535 0.11036479 1.93066676]
GLOBAL OPTIMUM:  29.78669648160634 GLOBAL OPT POSITION:  [-2.64026597 -0.82396314 -4.70497363]
GLOBAL OPTIMUM:  29.78669648160634 GLOBAL OPT POSITION:  [-2.64026597 -0.82396314 -4.70497363]
GLOBAL OPTIMUM:  29.78669648160634 GLOBAL OPT POSITION:  [-2.64026597 -0.82396314 -4.70497363]
GLOBAL OPTIMUM:  29.78669648160634 GLOBAL OPT POSITION:  [-2.64026597 -0.82396314 -4.70497363]
GLOBAL OPTIMUM:  29.78669648160634 GLOBAL OPT POSITION:  [-2.6402